<div align = "center">

# **Limpieza extra para Frecuencia-Severidad**

</div>

## Librerias

In [38]:
library(dplyr)
library(tidyr)

## Data

In [39]:
datos_limpios <- read.csv("../data/processed/datos_limpios.csv")

In [40]:
head(datos_limpios)

,Amparo,SumaDePagos,Modelo,Color,Carroceria,Referencia1,Referencia2,CLASE_FASECOLDA,TIPO_VEHICULO,SERVICIO,Sexo_Aseg,Edad,Desde,Hasta,Vr_Comercial,Pago,Accidentado,exposicion
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>
1,No aplica,0,2007_2013,Otros,SEDAN,Otros,Otros,AUTOMOVIL,Livianos,Particular,F,19_41,2011-09-29,2011-12-31,63430970,No,0,94
2,No aplica,0,2007_2013,Otros,SEDAN,Otros,Otros,AUTOMOVIL,Livianos,Particular,F,19_41,2012-01-01,2012-11-14,61150073,No,0,319
3,No aplica,0,2007_2013,PLATA,HATCHBACK,ATOS [2],PRIME,AUTOMOVIL,Livianos,Particular,M,63_85,2011-10-01,2011-12-31,18415081,No,0,92
4,No aplica,0,2007_2013,PLATA,HATCHBACK,ATOS [2],PRIME,AUTOMOVIL,Livianos,Particular,M,63_85,2012-01-01,2012-10-01,17752898,No,0,275
5,No aplica,0,2007_2013,PLATA,SEDAN,Otros,Otros,AUTOMOVIL,Livianos,Particular,F,19_41,2012-02-14,2012-12-31,37562534,No,0,322
6,No aplica,0,2007_2013,PLATA,SEDAN,Otros,Otros,AUTOMOVIL,Livianos,Particular,F,19_41,2013-01-01,2013-02-14,36667838,No,0,45


## Agrupando

In [41]:
# Variables categóricas relevantes para el modelo
variables_tarifa <- c("Modelo", "Color", "Carroceria", "CLASE_FASECOLDA", 
                      "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg", "Edad")

# Agregar datos por celda tarifaria
tabla_agregada <- datos_limpios %>%
  group_by(across(all_of(c(variables_tarifa)))) %>%
  summarize(
    valor = sum(Vr_Comercial),

    # Para el modelo de frecuencia
    n_siniestros = sum(Accidentado),
    exposicion_total = sum(exposicion),
    
    # Para el modelo de severidad
    suma_pagos = sum(SumaDePagos),
    n_observaciones = n(),
    
    .groups = 'drop'
  )

# Convertir variables categóricas a factores en tabla_agregada
tabla_agregada <- tabla_agregada %>%
  mutate(
    Color = as.factor(Color),
    Carroceria = as.factor(Carroceria),
    CLASE_FASECOLDA = as.factor(CLASE_FASECOLDA),
    TIPO_VEHICULO = as.factor(TIPO_VEHICULO),
    SERVICIO = as.factor(SERVICIO),
    Sexo_Aseg = as.factor(Sexo_Aseg),
    Edad = as.factor(Edad)
  )

# Crear subset para severidad (solo celdas con siniestros)
tabla_severidad <- tabla_agregada %>%
  filter(n_siniestros > 0) %>%
  mutate(
    severidad_media = suma_pagos / n_siniestros,
    # Mantener todos los niveles de los factores aunque no aparezcan en los datos filtrados
    across(where(is.factor), ~factor(., levels = levels(.)))
  )

In [42]:
head(tabla_agregada)

Modelo,Color,Carroceria,CLASE_FASECOLDA,TIPO_VEHICULO,SERVICIO,Sexo_Aseg,Edad,valor,n_siniestros,exposicion_total,suma_pagos,n_observaciones
<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<dbl>,<int>
1993_1999,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,F,19_41,101517338,2,2410,8014069,13
1993_1999,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,F,41_63,197695502,2,3862,2204046,22
1993_1999,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,19_41,180459141,4,4223,6450375,23
1993_1999,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,41_63,121930553,2,3077,4238816,16
1993_1999,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,63_85,115968579,0,2565,0,14
1993_1999,Otros,Otros,Otros,Livianos,Particular,F,41_63,79750398,2,1099,1208679,6


In [43]:
head(tabla_severidad)

Modelo,Color,Carroceria,CLASE_FASECOLDA,TIPO_VEHICULO,SERVICIO,Sexo_Aseg,Edad,valor,n_siniestros,exposicion_total,suma_pagos,n_observaciones,severidad_media
<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>
1993_1999,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,F,19_41,101517338,2,2410,8014069,13,4007034.6
1993_1999,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,F,41_63,197695502,2,3862,2204046,22,1102023.2
1993_1999,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,19_41,180459141,4,4223,6450375,23,1612593.7
1993_1999,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,41_63,121930553,2,3077,4238816,16,2119408.0
1993_1999,Otros,Otros,Otros,Livianos,Particular,F,41_63,79750398,2,1099,1208679,6,604339.6
1993_1999,Otros,SEDAN,AUTOMOVIL,Livianos,Particular,F,19_41,338559951,2,6132,8213167,35,4106583.6


In [44]:
# Primero, veamos qué tipo de variables tienes
str(tabla_agregada)

tibble [247 × 13] (S3: tbl_df/tbl/data.frame)
 $ Modelo          : chr [1:247] "1993_1999" "1993_1999" "1993_1999" "1993_1999" ...
 $ Color           : Factor w/ 4 levels "AMARILLO","BLANCO",..: 2 2 2 2 2 3 3 3 3 3 ...
 $ Carroceria      : Factor w/ 4 levels "CABINADO","HATCHBACK",..: 4 4 4 4 4 3 4 4 4 4 ...
 $ CLASE_FASECOLDA : Factor w/ 4 levels "AUTOMOVIL","AUTOMOVIL TAXI",..: 1 1 1 1 1 4 1 1 1 1 ...
 $ TIPO_VEHICULO   : Factor w/ 2 levels "Livianos","Pesados": 1 1 1 1 1 1 1 1 1 1 ...
 $ SERVICIO        : Factor w/ 4 levels "Oficial","Particular",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Sexo_Aseg       : Factor w/ 3 levels "F","M","No Binarie": 1 1 2 2 2 1 1 1 1 2 ...
 $ Edad            : Factor w/ 4 levels "19_41","41_63",..: 1 2 1 2 3 2 1 2 4 1 ...
 $ valor           : num [1:247] 1.02e+08 1.98e+08 1.80e+08 1.22e+08 1.16e+08 ...
 $ n_siniestros    : int [1:247] 2 2 4 2 0 2 2 6 4 4 ...
 $ exposicion_total: int [1:247] 2410 3862 4223 3077 2565 1099 6132 8985 3664 6334 ...
 $ suma_pagos      :

In [46]:
write.csv(tabla_severidad, "../data/processed/tabla_severidad.csv")